In [1]:
! pip install langchain langchainhub langchain-community chromadb beautifulsoup4 sentence-transformers huggingface-hub

In [2]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain import hub
from langchain_community.llms import Ollama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

### Indexing ###
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter()
splits = text_splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents = splits, embedding = embedding)
retriever = vectorstore.as_retriever()

### Retrieval and Generation ###

prompt = hub.pull("rlm/rag-prompt")

llm = Ollama(model = "gemma:2b")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm 
    | StrOutputParser()
)

response = rag_chain.invoke("What is Task Decomposition?")
print(response)

USER_AGENT environment variable not set, consider setting it to identify your requests.
/tmp/ipykernel_20905/1333729181.py:25: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
/home/arushkandare/Arush/New folder/RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event Client

Sure, here's a summary of the provided context about "What is Task Decomposition?":

**Task Decomposition** is a technique that helps an LLM decompose a complex task into smaller, simpler subtasks. This approach allows the LLM to handle tasks that are too complex to be processed by the model in its entirety.

**Chain of Thought (CoT)** is a framework that encourages an LLM to improve on its own outputs by explicitly presenting it with a sequence of past outputs, each annotated with feedback. This approach allows the model to self-reflect and refine its output over time.

**Reflection** is a process that allows an LLM to analyze its own performance and make improvements. This can be done through self-reflection (where the model is presented with a sequence of past outputs) or through human feedback.

**Task decomposition and reflection are used together in various approaches to LLM-powered autonomous agents. This approach allows the agents to learn from their past experiences and improv

In [3]:
response = rag_chain.invoke("What is in the document")
print(response)

The text does not provide any information about the game's data, view, and controller, which are crucial components for building an LLM-centered agent for the game.


In [4]:
response = rag_chain.invoke("What is a LLM")
response

"Sure, here's the answer to the question:\n\nAn LLM-powered autonomous agent system is a type of artificial intelligence that uses a large language model to control a set of agents or systems. This system can be used for various purposes, such as language translation, question answering, and problem solving.\n\nThe LLM functions as the brain of the system, and the agents are controlled by its actions. The system can also reflect on its past actions and use this information to improve its future behavior.\n\nThe system is composed of several components, including planning, reflection, and memory. The planning component breaks down complex tasks into smaller, manageable steps, while the reflection component allows the agent to learn from its past experiences. The memory component stores information that is used by the other components."

# INDEXING

In [24]:
question = "What kinds of pets do I like"
document = "My favourite pet is a cat"

In [25]:
query_result = embedding.embed_query(question)
document_result = embedding.embed_query(document)
len(query_result)

384

In [26]:
len(document_result)

384

In [27]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
similarity

np.float64(0.608624681647011)

In [28]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/home/arushkandare/Arush/New folder/RAG/XML.pdf")
pdf_docs = loader.load()

In [29]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50
)
splits = text_splitter.split_documents(pdf_docs)

In [30]:
vector_store = Chroma.from_documents(documents = splits, 
                                     embedding = embedding)
retriever = vector_store.as_retriever()

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


# RETRIEVAL

In [31]:
vector_store = Chroma.from_documents(documents = splits, 
                                     embedding = embedding)
Retriever = vector_store.as_retriever(search_kwargs = {"k": 1})

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [32]:
documents = Retriever.get_relevant_documents("What is XML?")

In [33]:
len(documents)

1

In [34]:
documents

[Document(metadata={'page_label': '1', 'author': 'santosh', 'source': '/home/arushkandare/Arush/New folder/RAG/XML.pdf', 'page': 0, 'creationdate': '2020-05-15T23:56:51-07:00', 'moddate': '2020-05-15T23:56:51-07:00', 'total_pages': 18, 'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013'}, page_content='XML is a software- and hardware-independent tool for storing and transporting data. \n \n\uf0b7 XML is a markup language much like HTML\uf020\uf020\n\uf0b7 XML was designed to store and transport data\uf020\uf020')]

# Generation

In [35]:
from langchain.prompts import ChatPromptTemplate

template = """ Answer the question based only on the following context: 
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template=' Answer the question based only on the following context: \n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [36]:
llm = Ollama(model = "gemma:2b")

In [37]:
chain = prompt | llm

In [43]:
documents

[Document(metadata={'page_label': '1', 'author': 'santosh', 'source': '/home/arushkandare/Arush/New folder/RAG/XML.pdf', 'page': 0, 'creationdate': '2020-05-15T23:56:51-07:00', 'moddate': '2020-05-15T23:56:51-07:00', 'total_pages': 18, 'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013'}, page_content='XML is a software- and hardware-independent tool for storing and transporting data. \n \n\uf0b7 XML is a markup language much like HTML\uf020\uf020\n\uf0b7 XML was designed to store and transport data\uf020\uf020')]

In [40]:
chain.invoke({"context": documents, "question": "What is XML?"})

'The context does not provide a definition for XML, so I cannot answer this question from the provided context.'

In [39]:
prompt_hub_rag = hub.pull("rlm/rag-prompt")
prompt_hub_rag

/home/arushkandare/Arush/New folder/RAG/.venv/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [46]:
rag_chain = (
    {"context": Retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What does this document consist of?")

'The document consists of a book with the following elements:\n\n- Title\n- Author\n- Price'